In [ ]:
!pip install transformers
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00


In [ ]:
!mkdir DETR_bounding_box_images
!mkdir DETR_Object_detection_output

In [ ]:
!unzip accident_frames.zip

Archive:  accident_frames.zip
   creating: accident_frames/
   creating: accident_frames/CLIP_183/
  inflating: __MACOSX/accident_frames/._CLIP_183  
   creating: accident_frames/CLIP_43/
  inflating: __MACOSX/accident_frames/._CLIP_43  
   creating: accident_frames/CLIP_88/
  inflating: __MACOSX/accident_frames/._CLIP_88  
   creating: accident_frames/CLIP_28/
  inflating: __MACOSX/accident_frames/._CLIP_28  
   creating: accident_frames/CLIP_311/
  inflating: __MACOSX/accident_frames/._CLIP_311  
   creating: accident_frames/CLIP_19/
  inflating: __MACOSX/accident_frames/._CLIP_19  
   creating: accident_frames/CLIP_124/
  inflating: __MACOSX/accident_frames/._CLIP_124  
   creating: accident_frames/CLIP_26/
  inflating: __MACOSX/accident_frames/._CLIP_26  
   creating: accident_frames/CLIP_89/
  inflating: __MACOSX/accident_frames/._CLIP_89  
   creating: accident_frames/CLIP_20/
  inflating: __MACOSX/accident_frames/._CLIP_20  
   creating: accident_frames/CLIP_298/
  inflating: __

In [ ]:
from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
from PIL import Image, ImageDraw, ImageFont
import os

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

preprocessor_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
directory_path = '/content/accident_frames/'
directory_files = os.listdir(directory_path)

# Use a default font
font = ImageFont.load_default()

for video in directory_files:
  print(video)
  if not video.startswith('.'):
    try:
      os.mkdir("/content/DETR_bounding_box_images/" + video.split(".")[0])
    except OSError as error:
      print(error)
    list_of_objects = []

    image_files = os.listdir(directory_path + "/" + video + "/")
    for image in image_files:
        image_path = os.path.join(directory_path + "/" + video + "/", image)
        pil_image = Image.open(image_path)
        object_count = {}
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # Make a copy of the image to draw bounding boxes without affecting the original clarity
        image_with_boxes = pil_image.copy()
        inputs = image_processor(images=pil_image, return_tensors="pt")
        outputs = model(**inputs)

        # convert outputs (bounding boxes and class logits) to COCO API
        target_sizes = torch.tensor([pil_image.size[::-1]])
        results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[
            0
        ]

        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            box = [round(i, 2) for i in box.tolist()]
            print(
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
            )
            predicted_label = model.config.id2label[label.item()]
            if(round(score.item(), 3) > 0.65):
                if predicted_label not in object_count:
                  object_count[predicted_label] = 0
                object_count[predicted_label] = object_count[predicted_label] + 1
        # Use a default font
        font = ImageFont.load_default()

        for score, label, bbox in zip(results["scores"], results["labels"], results["boxes"]):
          bbox = [int(coord) for coord in bbox]
          # Add a bounding box to the image
          draw = ImageDraw.Draw(image_with_boxes)
          draw.rectangle(bbox, outline='red', width=2)

          #font = ImageFont.truetype("path/to/your/font.ttf", 16)  # Provide the path to a TTF font file
          text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
          draw.rectangle(
              (bbox[0], bbox[1] - text_height, bbox[0] + text_width, bbox[1]),
              fill='red'
          )
          draw.text((bbox[0], bbox[1] - text_height), model.config.id2label[label.item()], fill='white', font=font)

        # Convert the PIL Image back to a NumPy array
        image_with_boxes.save("/content/DETR_bounding_box_images/" + video.split('.')[0] + "/" + image.split(".")[0] + ".png")
        list_of_objects.append({image.split(".")[0]:object_count})
    print(list_of_objects)
    f = open("/content/DETR_Object_detection_output/" + video.split(".")[0] + ".txt", "w")
    f.write(str(list_of_objects))
    f.close()

CLIP_127
[Errno 17] File exists: '/content/DETR_bounding_box_images/CLIP_127'
Detected car with confidence 0.994 at location [601.3, 356.14, 698.85, 404.31]


<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.934 at location [566.12, 361.88, 659.13, 406.97]
Detected traffic light with confidence 0.965 at location [495.14, 327.08, 525.89, 362.19]
Detected truck with confidence 0.901 at location [564.99, 361.58, 658.75, 407.32]


<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.932 at location [520.69, 366.5, 612.5, 405.42]


...

<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.994 at location [538.91, 360.72, 613.84, 404.62]
Detected car with confidence 0.998 at location [613.7, 358.35, 714.11, 403.72]


<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.998 at location [601.32, 356.75, 665.73, 398.84]
Detected car with confidence 0.91 at location [642.96, 358.16, 676.71, 388.32]
Detected car with confidence 0.998 at location [675.87, 352.63, 753.39, 387.22]
[{'frame60': {'car': 2}}, {'frame45': {'car': 2}}, {'frame30': {'car': 2}}, {'frame15': {'car': 3}}]


<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-12-d68a0f66d41a>:54: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


In [ ]:
!zip -r /content/DETR_bounding_box_images.zip /content/DETR_bounding_box_images
!zip -r /content/DETR_Object_detection_output.zip /content/DETR_Object_detection_output

  adding: content/DETR_bounding_box_images/ (stored 0%)
  adding: content/DETR_bounding_box_images/CLIP_127/ (stored 0%)
  adding: content/DETR_bounding_box_images/CLIP_127/frame60.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_127/frame45.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_127/frame15.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_127/frame30.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_308/ (stored 0%)
  adding: content/DETR_bounding_box_images/CLIP_308/frame29.png (deflated 1%)
  adding: content/DETR_bounding_box_images/CLIP_308/frame58.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_169/ (stored 0%)
  adding: content/DETR_bounding_box_images/CLIP_169/frame60.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_169/frame45.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP_169/frame15.png (deflated 0%)
  adding: content/DETR_bounding_box_images/CLIP